# 📘 텍스트 분할 (Text Splitter)

---

## 01. 텍스트 분할의 개념과 중요성. 다양한 전략의 활용

### 📌 개념
- LLM은 **Context Window** 제한이 있음 → 긴 문서를 그대로 입력 불가  
- **Text Splitter**를 사용해 긴 문서를 잘게 쪼개서 모델이 이해할 수 있게 함  
- 좋은 분할 전략은:
  - **문맥 보존** (자르더라도 의미 단위 유지)
  - **겹침(chunk_overlap)** 활용 → 앞뒤 문맥 손실 방지
  - 문서 포맷(코드, 마크다운, HTML 등)에 맞는 전용 Splitter 선택

---

## 02. CharacterTextSplitter

### 📌 개념
- **단순 문자 단위**로 분할
- 고정된 길이 기준 (예: 200자씩)

In [1]:
from langchain_text_splitters import CharacterTextSplitter

text = "이 문장은 CharacterTextSplitter를 이용해 잘려나갑니다."
splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=2)
chunks = splitter.split_text(text)

print(chunks)

['이 문장은 CharacterTextSplitter를 이용해 잘려나갑니다.']


## 03. RecursiveCharacterTextSplitter
### 📌 개념

- 문자/구두점/문단/줄바꿈 순서대로 분할을 시도

- 의미 단위 최대한 보존

- RAG 파이프라인에서 가장 많이 사용됨

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text = "LangChain은 LLM 활용을 돕는 프레임워크입니다.\n텍스트 스플리터는 RAG에서 핵심 요소입니다."
splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap=5)
chunks = splitter.split_text(text)

print(chunks)

['LangChain은 LLM 활용을', '활용을 돕는 프레임워크입니다.', '텍스트 스플리터는 RAG에서 핵심', '핵심 요소입니다.']


## 04. TokenTextSplitter
### 📌 개념

- 토큰 단위로 분할 (OpenAI tiktoken 사용)

- 토큰 단위 제어가 필요할 때 유용

In [3]:
from langchain_text_splitters import TokenTextSplitter

text = "이 문장은 토큰 단위로 분할됩니다."
splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=2)
chunks = splitter.split_text(text)

print(chunks)

['이 문장은', '은 토큰 �', '� 단위로', '�� 분할됩', '��니다.']


## 05. SemanticChunker
### 📌 개념

- 임베딩 기반 의미 단위 분할

- 단순 길이 대신 문장/단락 의미 유사도로 구분

- 성능: RAG 검색 품질 향상

In [2]:
# from langchain_experimental.text_splitter import SemanticChunker
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()
# splitter = SemanticChunker(embeddings)

# text = "LangChain은 다양한 기능을 제공합니다. 텍스트 분할은 핵심입니다."
# chunks = splitter.split_text(text)

# print(chunks)

## 06. Code Splitter
### 📌 개념

- 코드 파일 전용 분할기

- 함수, 클래스 단위로 잘라서 의미 보존

In [3]:
from langchain_text_splitters import PythonCodeTextSplitter

code = """
def add(a, b):
    return a + b

def multiply(a, b):
    return a * b
"""
splitter = PythonCodeTextSplitter(chunk_size=40, chunk_overlap=0)
chunks = splitter.split_text(code)

print(chunks)

['def add(a, b):\n    return a + b', 'def multiply(a, b):\n    return a * b']


## 07. MarkdownHeaderTextSplitter
### 📌 개념

- Markdown 헤더(#, ##, ###) 기준 분할

- 문서 구조(섹션 단위) 보존 가능

In [4]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

markdown_text = """
# 제목1
내용 A

## 소제목1
내용 B

## 소제목2
내용 C
"""

headers = [("#", "h1"), ("##", "h2")]
splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers)
chunks = splitter.split_text(markdown_text)

for c in chunks:
    print(c.page_content, c.metadata)

내용 A {'h1': '제목1'}
내용 B {'h1': '제목1', 'h2': '소제목1'}
내용 C {'h1': '제목1', 'h2': '소제목2'}


## 08. HTMLHeaderTextSplitter
### 📌 개념

- HTML 태그( h1, h2 ...) 기준으로 분할

- 웹페이지 문서 구조 보존 가능

In [5]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_text = """
<h1>메인 제목</h1>
<p>이것은 첫 번째 단락입니다.</p>
<h2>소제목</h2>
<p>두 번째 단락입니다.</p>
"""

headers = [("h1", "h1"), ("h2", "h2")]
splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers)
chunks = splitter.split_text(html_text)

for c in chunks:
    print(c.page_content, c.metadata)

메인 제목 {'h1': '메인 제목'}
이것은 첫 번째 단락입니다. {'h1': '메인 제목'}
소제목 {'h1': '메인 제목', 'h2': '소제목'}
두 번째 단락입니다. {'h1': '메인 제목', 'h2': '소제목'}


## 09. RecursiveJSONSplitter
### 📌 개념

- JSON 데이터를 재귀적으로 탐색하며 분할

- 계층적 JSON → LLM이 처리 가능한 작은 단위로 변환

In [6]:
from langchain_text_splitters import RecursiveJsonSplitter

json_data = {
    "id": 1,
    "name": "철수",
    "info": {
        "age": 25,
        "hobbies": ["독서", "등산", "게임"]
    },
    "history": [
        {"year": 2020, "event": "대학 입학"},
        {"year": 2024, "event": "졸업 예정"}
    ]
}

splitter = RecursiveJsonSplitter(max_chunk_size=50)
chunks = splitter.split_json(json_data)

for c in chunks:
    print(c)

{'id': 1, 'name': '철수', 'info': {'age': 25}}
{'info': {'hobbies': ['독서', '등산', '게임']}}
{'history': [{'year': 2020, 'event': '대학 입학'}, {'year': 2024, 'event': '졸업 예정'}]}


# ✅ 정리

- Character / RecursiveCharacter / Token : 길이/토큰 기반 기본 분할

- SemanticChunker : 임베딩 기반 의미 단위 분할

- Code Splitter : 코드 함수/클래스 단위 분할

- Markdown / HTML Splitter : 문서 구조 보존

- RecursiveJSONSplitter : JSON 계층적 분할

- 적절한 Splitter 선택 = RAG, 검색 성능 향상의 핵심